In [1]:
import torch
import torchtext
from torchtext.data import Field, Dataset, BPTTIterator
from torchtext.datasets import WikiText2
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math 
import numpy as np
from time import time

import re

Задача: используя библиотеку torchtext, сделать генератор данных для обучения сети, подобной рассмотренной в лекции.

In [2]:
# Создает токенайзер для разбиения текста на символы.
tokenize = lambda x: re.findall(".", x)
# Создает переменную класса torchtext.data.Field, хранящую информацию, необходимую для препроцессинга текста.
TEXT = Field(sequential=True, tokenize=tokenize, eos_token="<eos>", lower=True)
# Разбиение датасета WikiText2 на множества для обучения, валидации, проверки.
train, valid, test = WikiText2.splits(TEXT)
# Построение словаря.
TEXT.build_vocab(train, vectors="glove.6B.200d")
# Проверка словаря. Каждому символу присвоено числовое значение.
print("Длина словаря: ", len(list(TEXT.vocab.stoi.items())))
print(list(TEXT.vocab.stoi.items())[:30])

Длина словаря:  245
[('<unk>', 0), ('<pad>', 1), ('<eos>', 2), (' ', 3), ('e', 4), ('t', 5), ('a', 6), ('n', 7), ('i', 8), ('o', 9), ('r', 10), ('s', 11), ('h', 12), ('d', 13), ('l', 14), ('u', 15), ('c', 16), ('m', 17), ('f', 18), ('g', 19), ('p', 20), ('w', 21), ('b', 22), ('y', 23), ('k', 24), (',', 25), ('.', 26), ('v', 27), ('<', 28), ('>', 29)]


In [3]:
# Инициализация переменных
batch_size = 128
sequence_length = 30

In [4]:
# Создает BPTTIterator для разбиения корпуса на последовательные батчи с таргетом, сдвинутым на 1
train_iter, valid_iter, test_iter = BPTTIterator.splits((train, valid, test),
                                                         batch_size=batch_size,
                                                         bptt_len=sequence_length,    
                                                         shuffle=True)

In [5]:
# Инициализация переменных
eval_batch_size = 128
grad_clip = 0.1
lr = 4.
best_val_loss = None
log_interval = 100

weight_matrix = TEXT.vocab.vectors
ntokens = weight_matrix.shape[0]
nfeatures = weight_matrix.shape[1]

In [6]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5, lnorm=False):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp) # (длина словаря, количество признаков)        
        self.lnorm = None
        if lnorm:
            self.lnorm = nn.LayerNorm(ninp)
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)  # (кол-во признаков, скрытых состояний, слоев)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken) # (переводит из признаков в токен словаря)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        # Инициализация весов Embedding и FC
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, hidden=None):
        emb = self.drop(self.encoder(x)) # (Выход: длина последовательности, batch_size, кол-во признаков)
        if self.lnorm is not None:
            emb = self.lnorm(emb)
        # (Размерность входа соответствует)
        output, hidden = self.rnn(emb, hidden) 
        # (output: длина последовательности, batch_size, кол-во скрытых)
        # (hidden: 2 * (Кол-во слоев, batch_size, кол-во скрытых). 1-й h_n, 2-й c_n)
        output = self.drop(output)
        # (Вход: N, кол-во признаков (скрытых))
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        # (Выход: N, размер словаря)
        # Возвращает (длина последовательности, batch_size, длина словаря)
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        # Инициализация скрытого состояния
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (weight.new(self.nlayers, bsz, self.nhid).zero_(),
                    weight.new(self.nlayers, bsz, self.nhid).zero_())
        else:
            return weight.new(self.nlayers, bsz, self.nhid).zero_()

In [7]:
# Валидация.
def evaluate(data_iter):
    model.eval()
    total_loss = 0    
    # Инициализирует hidden. Не ислользуется?
    hidden = model.init_hidden(eval_batch_size)
    for i, batch_data in enumerate(data_iter):
        # torchtext.data.BPTTIterator возвращает данные в таком формате:
        data, targets = batch_data.text, batch_data.target
        # Получает результат.
        output, hidden = model(data)
        output_flat = output.view(-1, ntokens)
        # Накапливает ошибку.
        total_loss += criterion(output_flat, targets.view(-1)).item()
    # Возвращает среднюю ошибку.
    return total_loss / len(data_iter)

In [8]:
# Обучение.
def train():
    model.train()
    total_loss = 0    
    for batch, batch_data in enumerate(train_iter):
        data, targets = batch_data.text, batch_data.target        
        # Обнуление градиента между батчами.
        #model.zero_grad()
        optimizer.zero_grad()
        # Получение выхода модели.
        output, hidden = model(data)        
        # Расчет и обратное распространение ошибки.
        loss = criterion(output.view(-1, ntokens), targets.view(-1))
        loss.backward()

        # Обновление весов.
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        #for p in model.parameters():
        #    p.data.add_(-lr, p.grad.data)            
        optimizer.step()

        # Накопление ошибки
        total_loss += loss.item()

        # Вывод средней ошибки по log_interval.
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_iter), lr, cur_loss, math.exp(cur_loss)))
            total_loss = 0

In [12]:
# Создание модели.
# (self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5)
model = RNNModel('LSTM', ntokens, 128, 128, 2, 0.3, lnorm=True)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adadelta(model.parameters(), lr=1e+2, weight_decay=0.)

Эксперименты с оптимизаторами показали лучший результат после 1й эпохи для Adadelta с lr=100 (loss около 1.7). 2 слоя оказались лучше, чем 1 и 3 (тоже после 1й эпохи). GRU не удалось превзойти результат LSTM. Добавление LayerNormalization немного уменьшило ошибку, и сложилось впечатление, что после первой эпохи стали появляться более осмысленные слова. Возможно, случайность.

In [10]:
# Генерация последовательности.
def generate(n=50, temp=1.):
    model.eval()
    # Генерация начального символа.
    x = torch.rand(1, 1).mul(ntokens).long()
    hidden = None
    out = []
    # Создание последовательности длины n.
    for i in range(n):
        output, hidden = model(x, hidden)
        # Получает распределение на следующую букву.
        s_weights = output.squeeze().data.div(temp).exp()
        # Сэмплирует индекс из распределения и принимает за следующий.
        s_idx = torch.multinomial(s_weights, 1)[0]
        x.data.fill_(s_idx)
        # Переводит индекс в букву и дополняет последовательность.
        s = TEXT.vocab.itos[s_idx]
        out.append(s)
    # Возвращает строку.
    return ''.join(out)

In [13]:
# Обучение, валидация и вывод результатов.
with torch.no_grad():
    print('sample:\n', generate(50), '\n')

for epoch in range(1, 6):
    start_time = time()
    train()
    val_loss = evaluate(valid_iter)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')
    print("Epoch time: {}".format((time()-start_time)/60.))


sample:
 攻q–ṃoử–⁄αóóắァơcāấq0隊่჻5≤оşоμ°̃bตbʿ£е่隊¡₤aịầァ3'þdx機 

| epoch   1 |   100/ 2808 batches | lr 4.00 | loss  2.97 | ppl    19.55
| epoch   1 |   200/ 2808 batches | lr 4.00 | loss  2.19 | ppl     8.94
| epoch   1 |   300/ 2808 batches | lr 4.00 | loss  2.07 | ppl     7.92
| epoch   1 |   400/ 2808 batches | lr 4.00 | loss  2.00 | ppl     7.41
| epoch   1 |   500/ 2808 batches | lr 4.00 | loss  1.95 | ppl     7.04
| epoch   1 |   600/ 2808 batches | lr 4.00 | loss  1.92 | ppl     6.84
| epoch   1 |   700/ 2808 batches | lr 4.00 | loss  1.90 | ppl     6.68
| epoch   1 |   800/ 2808 batches | lr 4.00 | loss  1.88 | ppl     6.55
| epoch   1 |   900/ 2808 batches | lr 4.00 | loss  1.86 | ppl     6.46
| epoch   1 |  1000/ 2808 batches | lr 4.00 | loss  1.85 | ppl     6.37
| epoch   1 |  1100/ 2808 batches | lr 4.00 | loss  1.84 | ppl     6.30
| epoch   1 |  1200/ 2808 batches | lr 4.00 | loss  1.83 | ppl     6.25
| epoch   1 |  1300/ 2808 batches | lr 4.00 | loss  1.82 | ppl     6.17
| 

| epoch   4 |  1600/ 2808 batches | lr 4.00 | loss  1.70 | ppl     5.46
| epoch   4 |  1700/ 2808 batches | lr 4.00 | loss  1.70 | ppl     5.45
| epoch   4 |  1800/ 2808 batches | lr 4.00 | loss  1.70 | ppl     5.45
| epoch   4 |  1900/ 2808 batches | lr 4.00 | loss  1.71 | ppl     5.53
| epoch   4 |  2000/ 2808 batches | lr 4.00 | loss  1.69 | ppl     5.42
| epoch   4 |  2100/ 2808 batches | lr 4.00 | loss  1.70 | ppl     5.48
| epoch   4 |  2200/ 2808 batches | lr 4.00 | loss  1.70 | ppl     5.50
| epoch   4 |  2300/ 2808 batches | lr 4.00 | loss  1.71 | ppl     5.51
| epoch   4 |  2400/ 2808 batches | lr 4.00 | loss  1.69 | ppl     5.45
| epoch   4 |  2500/ 2808 batches | lr 4.00 | loss  1.69 | ppl     5.44
| epoch   4 |  2600/ 2808 batches | lr 4.00 | loss  1.71 | ppl     5.53
| epoch   4 |  2700/ 2808 batches | lr 4.00 | loss  1.70 | ppl     5.47
| epoch   4 |  2800/ 2808 batches | lr 4.00 | loss  1.69 | ppl     5.43
----------------------------------------------------------------

После 5 эпох ошибка так и не смогла дальше снизится. Конечно, нужно было подбирать параметры на более, чем одной эпохе, но очень уж медленный pytorch. Осталось пространство для экспериментов. Интересно еще попробовать последовательность слов.